In [75]:
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf
from tensorflow.keras.models import load_model

In [76]:
## Load the trained model, pickle file
model = load_model('model.h5')

with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('onehot_encoder_geo.pkl', 'rb') as file:
    onehot_encoder_geo = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)



In [77]:
# Example Input Data
input_data = {
    'CreditScore':600,
    'Geography':'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary': 50000
}

In [78]:
input_data = pd.DataFrame([input_data])
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [79]:
input_data['Gender'] = label_encoder_gender.transform(input_data['Gender'])
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [80]:
geo_encoded = onehot_encoder_geo.transform([input_data['Geography']])
geo_encoded_df = pd.DataFrame(geo_encoded.toarray(), columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

geo_encoded_df

/Users/shubhamgangwar/Documents/Python/AI_ML Krish Nayak/AI_ML Krish/venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [81]:
## Combining the dataset with the encoded values
input_data = pd.concat([input_data.drop('Geography', axis=1), geo_encoded_df], axis=1)
input_data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [82]:
## Scaling the data
input_scaled = scaler.transform(input_data)

In [83]:
## Predict Churn
prediction = model.predict(input_scaled)
prediction
 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step


array([[0.01350371]], dtype=float32)

In [84]:
pred_probablily = prediction[0][0]
pred_probablily

0.013503713

In [85]:
if pred_probablily > 0.5:
    print('The customer is likely to churn')
else:
    print('The customer is not likely to churn')

The customer is not likely to churn
